In [ ]:
import sys
import os
import errno
from datetime import datetime
import warnings
import time as tm
import json

import pandas as pd
pd.options.display.max_rows = 300
pd.options.display.max_columns = 100
import numpy as np
import s3fs
import PyPDF2
import plotly as py
import plotly.graph_objects as go

### Upload and prepare datasets

#### AWS S3 load

In [ ]:
def get_df(file_name):
    fs = s3fs.S3FileSystem(profile_name = 'dmitry.zorikhin@XXX')
    df = pd.read_csv(fs.open(f'discover.energy-datasource/XXX/{file_name}'))
    return df

In [ ]:
assets_df = get_df('assets.csv')

basins_df = get_df('basins.csv')

assets_basins_df = assets_df.merge(basins_df, left_on=['basin_id'], 
                                   right_on=['id'], how='left', suffixes=('', '_basin'))

asset_production_df = get_df('asset_quarterly_data.csv')

assets_basins_dataset = asset_production_df.merge(assets_basins_df, left_on=['asset_id'], 
                                                  right_on=['id'], how='left', 
                                                  suffixes=('', '_asset'))
(assets_basins_dataset
 .loc[(assets_basins_dataset.name_basin == 'ETX/Haynesville'),
      'name_basin']) = 'East TX Haynesville' 

asset_data_df = get_df('asset_metrics_data.csv')

In [ ]:
list(assets_basins_dataset.name_basin.unique())

### Declare constants

In [ ]:
BASIN_NAME = 'Permian'
PERMIAN_TYPE = ''

"""Permian should be divided in 3 parts which will be set as parameters:
large_diversified_ - big companies
us_oil_ - medium size companies
micro_cap_ - small size companies
Underscore is needed for proper file name
For now this script should be executed 3 times for each parameter until company level section

"""

if BASIN_NAME == 'Permian':
    PERMIAN_TYPE = 'us_oil_'
    
    large_diversified = ['APA', 'COP', 'EOG', 'MRO', 'NBL', 'OXY', 'PXD', 'XOM']
    us_oil = ['CXO', 'DVN', 'FANG', 'MTDR', 'OVV', 'PDCE', 'PE', 'SM', 'WPX', 'XEC']
    micro_cap = ['CDEV', 'CPE', 'OAS', 'QEP']
    
    assets_basins_dataset = assets_basins_dataset[(assets_basins_dataset.ticker.isin(us_oil))]
    assets_count = (assets_basins_dataset[(assets_basins_dataset.name_basin == 'Permian') 
                                          & (assets_basins_dataset.ticker.isin(us_oil))]
                    .asset_id.nunique())

In [ ]:
REPORT_PERIOD = '2020_Q1'
START_QUARTER_ACTUAL = 20192
END_QUARTER_ACTUAL = 20201
START_QUARTER_FORECAST = 20202
END_QUARTER_FORECAST = 20212
COLORS = [
    '#91bd3a', '#64c4ed', '#5f6caf', '#cfb495', '#deff8b',
    '#faafff', '#b9cced', '#347474', '#be8abf', '#fddb3a',
    '#e58a8a', '#75b79e', '#7f78d2', '#f1935c', '#d9bf77',
    '#50bda1', '#ffd082', '#35495e', '#7fa998', '#a8d3da'
] * 10 

In [ ]:
TODAY = datetime.now().strftime('%d-%m-%Y')
TODAY

In [ ]:
path = f'../reports/{REPORT_PERIOD}'

try:
    os.mkdir(path)
    os.mkdir(f'{path}/{BASIN_NAME}/')
    print(f'Folders {REPORT_PERIOD} and {REPORT_PERIOD}/{BASIN_NAME} created')
except OSError as e:
    if e.errno == errno.EEXIST:
        print(f'Folder {REPORT_PERIOD} already exists')
    try:
        os.mkdir(f'{path}/{BASIN_NAME}/')
        print(f'Folder {REPORT_PERIOD}/{BASIN_NAME} created')
    except OSError as e:
        if e.errno == errno.EEXIST:
            print(f'Folder {REPORT_PERIOD}/{BASIN_NAME} already exists')
        else:
            raise

In [ ]:
try:
    os.mkdir(f'{path}/{BASIN_NAME}/pdf')
    print(f'Folder {REPORT_PERIOD}/{BASIN_NAME}/pdf created')
except OSError as e:
    if e.errno == errno.EEXIST:
        print(f'Folder {REPORT_PERIOD}/{BASIN_NAME}/pdf already exists')

### Declare functions

#### Parameter per basin and company function

In [ ]:
def dataset_for_analysis_basin_level(prepared_dataset, parameter, name_basin='', 
                                     aggregate='sum', aggregate_basis='basin', 
                                     start_quarter='', end_quarter=''):
    """Inputs:
    prepared_dataset - assets_basins_dataset as default, prepared above
    parameter - values to analyse (assets_basins_dataset columns)
    name_basin - basin name to analyse, default is data for all basins
    aggregate - sum as default
    aggregate_basis - aggregation basen on basin level or company level in result dataset
    start_quarter - choose start quarter for analysis, default value is empty (no restriction 
    to start quarter)
    end_quarter - choose end quarter for analysis, default value is empty (no restriction 
    to end quarter)
    
    Return:
    pivot table with aggregates for parameter across basin(s) or companies with or without 
    timeline restriction
    
    """
    df = (prepared_dataset
          .groupby(['name_basin', 'quarter', 'ticker', 'name'])
          .agg({parameter: aggregate})
          .reset_index())
    
    if name_basin != '':
        df = df[df['name_basin'] == name_basin]
    
    if start_quarter != '' and end_quarter != '':
        df = df[(df['quarter'] >= start_quarter) & (df['quarter'] <= end_quarter)]
    
    if aggregate_basis == 'company':
        df_pivot = pd.pivot_table(df, index=['ticker', 'name_basin', 'name'], 
                                  columns='quarter', values=parameter)
    else:
        df_pivot = pd.pivot_table(df, index=['name_basin', 'ticker', 'name'], 
                                  columns='quarter', values=parameter)
    
    return df_pivot

In [ ]:
def get_pie_plot_settings(df, column, title, path):
    
    data = [{ 'type': "pie", 
              'labels': df['Company'].tolist(), 
              'values': df[column].tolist(),
              'textinfo': 'value'
            }]
    layout = { 'title': title,
               'legend': {'orientation': 'h', 'xanchor': 'center', 'x': 0.5},
               'colorway': COLORS,
               'height': 600}
    
    if '/' in title:
        title = title.replace('/', ' per ')
        
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {str(layout)};\n'
           ) 

In [ ]:
def get_simple_bar_chart_settings(df, column, title, path):
    
    data = [{ 'type': 'bar',
              'x': df['asset_name'].tolist(), 
              'y': df[column].tolist()
            }]
    layout = { 'xaxis': {'title': 'Basin Assets', 'automargin': True},
               'title': title,
               'yaxis': {'title': column, 'ticks': 'outside'},
               'colorway': COLORS,
               'height': 800}
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    layout = str(layout).replace('True', 'true')
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {layout};\n'
            f'\nconfig = {config};\n'
           ) 

In [ ]:
def get_bubble_chart_settings(df, title, path):
    
    hover_text = []

    for index, row in df.iterrows():
        hover_text.append(('Company: {ticker}<br>'
                          'Asset Name: {name}<br>'
                          'Capex ($MM): {capex}<br>'
                          'Net Production (Mboe/d): {net_prod}<br>').format(
                                                ticker=row['ticker'],
                                                name=row['name'],
                                                capex=row['Capex ($MM)'],
                                                net_prod=row['Net Production (Mboe/d)']))    
    
    df['text'] = hover_text
    
    data = []
    
    for ticker in df.ticker.unique():
        data.append(
            {'type': 'scatter',
             'name': ticker,
              'x': df[df['ticker'] == ticker]['Capex ($MM)'].tolist(), 
              'y': df[df['ticker'] == ticker]['Net Production (Mboe/d)'].tolist(),
              'mode': 'markers',
              'marker': {'size': 30},
             'hoverinfo': 'text',
             'hovertext': df[df['ticker'] == ticker]['text'].tolist()
            }
        )
    
    layout = { 'xaxis': {'title': 'Net Capex ($MM)'},
               'title': title,
               'yaxis': {'title': 'Net Production (Mboe/d)'},
               'legend': {'itemclick': 'toggleothers', 'itemdoubleclick': 'toggle'},
               'colorway': COLORS,
               'height': 700}
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    layout = str(layout).replace('True', 'true')
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {layout};\n'
            f'\nconfig = {config};\n'
           )

In [ ]:
def get_horizontal_stack_bar_chart_settings(df, title, path):
    
    data = [{'type': 'bar',
             'name': BASIN_NAME,
             'y': df['Company'].tolist(),
             'x': df[f'{BASIN_NAME}_p'].tolist(),          
             'orientation': 'h'},
            {'type': 'bar',
             'name': 'Other basins',
             'y': df['Company'].tolist(),
             'x': df['Other_basins_p'].tolist(),          
             'orientation': 'h'}]

    layout = { 'barmode': 'stack',
               'title': title,
               'xaxis': {'tickformat': '.0%', 'ticks': 'outside'},
               'yaxis': {'autorange': 'reversed', 'ticks': 'outside'},
               'legend': {'orientation': 'h', 'x': 0.35, 'itemclick': 'toggleothers', 
                          'itemdoubleclick': 'toggle'},
               'colorway': COLORS,
               'height': 600}
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {str(layout)};\n'
            f'\nconfig = {config};\n'
           ) 
    

In [ ]:
def hex_to_rgb(hex_color: str):
    
    hex_color = hex_color.lstrip("#")
    if len(hex_color) == 3:
        hex_color = hex_color * 2
        
    return int(hex_color[0:2], 16), int(hex_color[2:4], 16), int(hex_color[4:6], 16)

In [ ]:
def get_stacked_chart_settings(df, title, type_hc, path, parameter=''):  # Stacked
    
    # Check if all values are zeros, do not need to display the chart
    if df.iloc[:, 1:].sum().values.sum() == 0: 
        return ('data : none')
    
    data = []
    
    assets = df.columns[1:].tolist()
    assets_max_value_name = max([len(item) for item in assets])
    assets_number = len(df.columns[1:].tolist())
    
    if parameter == 'general' and assets_max_value_name >= 38:
        fontsize = 9
    elif parameter == 'general' and assets_max_value_name > 32 and assets_max_value_name < 38:
        fontsize = 11
    else:
        fontsize = 12

    for asset_id in range(assets_number):
        data.append(
            {'type': 'scatter',
             'name': assets[asset_id],
             'y': df[assets[asset_id]].tolist(),
             'x': list(range(len(df['quarter']))),
             'hoverinfo': 'name+x+y',
              'mode': 'lines',
             'line': {'width': 0.1},
             'fillcolor': f'rgba{(*hex_to_rgb(COLORS[asset_id]), 1.0)}',
             'stackgroup': 'one'}
        )
    
    if type_hc == 'oil':
        y_axis_label = 'MMbbl/d'
    elif type_hc == 'gas':
        y_axis_label = 'MMcf/d'
    
    if BASIN_NAME == 'Permian' and parameter == 'general' and assets_count > 26:
        layout = { 'title': title,
                   'xaxis': {'title': 'Quarter', 'ticks': 'outside', 'tickmode': 'array', 
                             'tickvals': list(range(len(df['quarter']))), 
                             'ticktext': xlabels_format(df), 'ticklen': 8},
                   'yaxis': {'ticks': 'outside', 'title': {'text': y_axis_label, 'standoff': 15}},
                   'legend': {'orientation': 'v', 'itemclick': 'toggleothers', 
                              'itemdoubleclick': 'toggle'},
                   'colorway': COLORS,
                   'height': 900}
        
        fig = go.Figure(data=data, layout={ 'title': title,
                                            'xaxis': {'title': 'Quarter', 'ticks': 'outside', 
                                                      'tickmode': 'array', 
                                                      'tickvals': list(range(len(df['quarter']))), 
                                                      'ticktext': xlabels_format(df), 
                                                      'ticklen': 8},
                                            'yaxis': {'ticks': 'outside', 
                                                      'title': {'text': y_axis_label, 
                                                                'standoff': 15}},                                            
                                            'colorway': COLORS,
                                            'showlegend': False,
                                            'height': 700})
        fig.write_image(f'{path}/pdf/{title}.pdf')
        
    else:        
        layout = {'title': title,
                  'xaxis': {'title': 'Quarter', 'ticks': 'outside', 'tickmode': 'array', 
                            'tickvals': list(range(len(df['quarter']))), 
                            'ticktext': xlabels_format(df), 'ticklen': 8},
                  'yaxis': {'ticks': 'outside', 'title': {'text': y_axis_label, 'standoff': 15}},
                  'showlegend': True,
                  'legend': {'orientation': 'h', 'itemclick': 'toggleothers', 
                             'itemdoubleclick': 'toggle', 'xanchor': 'center', 
                             'x': 0.5, 'y': -0.11},
                  'colorway': COLORS,
                  'height': 900}
        
        fig = go.Figure(data=data, layout={'title': title,
                                           'xaxis': {'title': 'Quarter', 'ticks': 'outside', 
                                                     'tickmode': 'array', 
                                                     'tickvals': list(range(len(df['quarter']))), 
                                                     'ticktext': xlabels_format(df), 
                                                     'ticklen': 8},
                                           'yaxis': {'ticks': 'outside', 
                                                     'title': {'text': y_axis_label, 
                                                               'standoff': 15}},
                                           'showlegend': True,
                                           'legend': {'font': {'size': fontsize}, 
                                                      'orientation': 'h', 
                                                      'itemclick': 'toggleothers', 
                                                      'itemdoubleclick': 'toggle', 
                                                      'xanchor': 'center', 'x': 0.5, 'y': -0.11},
                                           'colorway': COLORS,
                                           'height': 900})
        fig.write_image(f'{path}/pdf/{title}.pdf')
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    layout = str(layout).replace('True', 'true')
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {layout};\n'
            f'\nconfig = {config};\n'
           )

In [ ]:
def get_stacked_bar_chart_settings(df, title, type_values, path, parameter=''):
    
    if df.iloc[:, 1:].sum().values.sum() == 0:
        return ('data : none')
    
    data = []
    
    assets = df.columns[1:].tolist()
    assets_max_value_name = max([len(item) for item in assets])
    assets_number = len(df.columns[1:].tolist())
    
    if parameter == 'general' and assets_max_value_name >= 38:
        fontsize = 9
    elif parameter == 'general' and assets_max_value_name > 32 and assets_max_value_name < 38:
        fontsize = 11
    else:
        fontsize = 12

    for asset_id in range(assets_number):
        data.append(
            {'type': 'bar',
             'name': assets[asset_id],
             'y': df[assets[asset_id]].tolist(),
             'x': list(range(len(df['quarter']))),
             'hoverinfo': 'name+y'}
        )
    
    if type_values == 'rig':
        y_axis_label = 'Active rigs'
    elif type_values == 'well':
        y_axis_label = 'New Wells Brought Online'
    elif type_values == 'capex':
        y_axis_label = '$MM'

    if BASIN_NAME == 'Permian' and parameter == 'general' and assets_count > 26:
        layout = { 'barmode': 'stack',
                   'title': title,
                   'xaxis': {'title': 'Quarter', 'ticks': 'outside', 'tickmode': 'array', 
                             'tickvals': list(range(len(df['quarter']))), 
                             'ticktext': xlabels_format(df), 'ticklen': 8},
                   'yaxis': {'ticks': 'outside', 'title': y_axis_label},
                   'legend': {'orientation': 'v', 'itemclick': 'toggleothers', 
                              'itemdoubleclick': 'toggle'},
                   'colorway': COLORS,
                   'height': 900 }
        
        fig = go.Figure(data=data, layout={ 'barmode': 'stack',
                                            'title': title,
                                            'xaxis': {'title': 'Quarter', 'ticks': 'outside', 
                                                      'tickmode': 'array', 
                                                      'tickvals': list(range(len(df['quarter']))), 
                                                      'ticktext': xlabels_format(df), 
                                                      'ticklen': 8},
                                            'yaxis': {'ticks': 'outside', 'title': y_axis_label},                                            
                                            'colorway': COLORS,
                                            'showlegend': False,
                                            'height': 700 })
        fig.write_image(f'{path}/pdf/{title}.pdf')
    
    else:    
        layout = {'barmode': 'stack',
                  'title': title,
                  'xaxis': {'title': 'Quarter', 'ticks': 'outside', 'tickmode': 'array', 
                            'tickvals': list(range(len(df['quarter']))), 
                            'ticktext': xlabels_format(df), 'ticklen': 8},
                  'yaxis': {'ticks': 'outside', 'title': y_axis_label},
                  'showlegend': True,
                  'legend': {'orientation': 'h', 'itemclick': 'toggleothers', 
                             'itemdoubleclick': 'toggle', 'xanchor': 'center', 
                             'x': 0.5, 'y': -0.11},
                  'colorway': COLORS,
                  'height': 900}
        
        fig = go.Figure(data=data, layout={'barmode': 'stack',
                                           'title': title,
                                           'xaxis': {'title': 'Quarter', 'ticks': 'outside', 
                                                     'tickmode': 'array', 
                                                     'tickvals': list(range(len(df['quarter']))), 
                                                     'ticktext': xlabels_format(df), 
                                                     'ticklen': 8},
                                           'yaxis': {'ticks': 'outside', 'title': y_axis_label},
                                           'showlegend': True,
                                           'legend': {'font': {'size': fontsize}, 
                                                      'orientation': 'h', 
                                                      'itemclick': 'toggleothers', 
                                                      'itemdoubleclick': 'toggle', 
                                                      'xanchor': 'center', 'x': 0.5, 'y': -0.11},
                                           'colorway': COLORS,
                                           'height': 900})
        fig.write_image(f'{path}/pdf/{title}.pdf')
    
    config = {'displayModeBar': True,
              'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    layout = str(layout).replace('True', 'true')
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {layout};\n'
            f'\nconfig = {config};\n'
           )

In [ ]:
def get_grouped_bar_chart_settings(df, title, path):
    
    data = [{ 'type': 'bar',
             'name': f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q fact',
              'x': df['ticker'].tolist(), 
              'y': df[f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q fact'].tolist()
            },
            { 'type': 'bar',
             'name': f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q forecast',
              'x': df['ticker'].tolist(), 
              'y': df[f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q forecast'].tolist()
            }
           ]

    layout = { 'barmode': 'group',           
               'title': title,
               'xaxis': {'ticks': 'outside'},
               'yaxis': {'title': 'New Wells Brought Online', 'ticks': 'outside'},
               'legend': {'orientation': 'h', 'itemclick': 'toggleothers', 
                          'itemdoubleclick': 'toggle', 
                          'xanchor': 'center', 'x': 0.5, 'y': -0.10},
               'colorway': COLORS,
               'height': 700}
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    if '/' in title:
        title = title.replace('/', 'and')
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    layout = str(layout).replace('True', 'true')
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {layout};\n'
            f'\nconfig = {config};\n'
           )

In [ ]:
def get_horizontal_bar_chart_settings(df, title, type_chart, path):
    
    if type_chart == 'lateral':
        x_axis_label = 'Lateral length, ft'
        column = 'value'
    elif type_chart == 'distance':
        x_axis_label = 'Feet between wells, ft'
        column = 'value'
    elif type_chart == 'efficiency':
        x_axis_label = 'Net Production (Mboe/d)'
        column = 'Net Production (Mboe/d)'
    
    data = [{ 'type': 'bar',
              'x': df[column].tolist(),
              'y': df['asset_name'].tolist(),          
              'orientation': 'h'
           }]
    

    layout = {
               'title': title,
               'xaxis': {'title': x_axis_label, 'tickformat': '.0'},
               'yaxis': {'autorange': 'reversed', 'ticks': 'outside'},
               'height': 900, 'width': 900,
               'colorway': COLORS,
               'margin': {'l': 250}
             }
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {str(layout)};\n'
            f'\nconfig = {config};\n'
           ) 
    

In [ ]:
def get_line_chart_settings(df, title, type_hc, path):
    
    if df.empty: # check if values are absent, do not need to display the chart
        return ('data : none')
    
    data = []

    for asset in df.columns[1:].tolist():
        if df[asset].isnull().all():
            continue
        else:
            data.append({ 'type': 'scatter',
                     'name': asset,
                      'x': list(range(len(df['quarter']))), 
                      'y': df[asset].tolist(),
                      'mode': 'lines+markers',
                      'hoverinfo': 'name+y'
                    })

    if type_hc == 'oil':
        y_axis_label = 'bbl/d'
    elif type_hc == 'gas':
        y_axis_label = 'Mcf/d'
        
    layout = {            
               'title': title,
               'xaxis': {'showline': True, 'title': 'Quarter', 'ticks': 'outside', 
                         'tickmode': 'array', 'tickvals': list(range(len(df['quarter']))), 
                         'ticktext': xlabels_format(df), 'ticklen': 8},
               'yaxis': {'ticks': 'outside', 'title': {'text': y_axis_label, 'standoff': 10}, 
                         'tickformat': '.0', 'rangemode': 'tozero'},
               'showlegend': True,
               'legend': {'orientation': 'h', 'itemclick': 'toggleothers', 
                          'itemdoubleclick': 'toggle', 'xanchor': 'center', 'x': 0.5, 'y': -0.13},
               'colorway': COLORS,
               'height': 700, 'width': 950}
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    layout = str(layout).replace('True', 'true')
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {layout};\n'
            f'\nconfig = {config};\n'
           )
    

In [ ]:
def get_horizontal_grouped_bar_chart_settings(df, title, path):
    
    data = [{ 'type': 'bar',
             'name': 'Lateral length, ft',
              'x': df['Lateral length, ft'].tolist(), 
              'y': df['asset'].tolist(),
             'orientation': 'h'
            },
            { 'type': 'bar',
             'name': 'Feet between wells, ft',
              'x': df['Feet between wells, ft'].tolist(), 
              'y': df['asset'].tolist(),
             'orientation': 'h'
            }
           ]


    layout = { 'barmode': 'group',           
               'title': title,
               'xaxis': {'tickformat': '.0'},
               'yaxis': {'ticks': 'outside', 'ticklen': 8, 'autorange': 'reversed'},
               'legend': {'orientation': 'h', 'itemclick': 'toggleothers', 
                          'itemdoubleclick': 'toggle', 'xanchor': 'center', 'x': 0.5},
               'height': 700, 'width': 900,
               'colorway': COLORS,
               'margin': {'l': 250}}
    
    config = { 'displayModeBar': True,
               'modeBarButtonsToRemove': [ 'sendDataToCloud',
                                           'lasso2d',
                                           'zoomIn2d',
                                           'zoomOut2d',
                                           'select2d',
                                           'resetScale2d',
                                           'toggleSpikelines' ],
                'displaylogo': False,
                'showTips': True}
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    config = str(config).replace('True', 'true').replace('False', 'false')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {str(layout)};\n'
            f'\nconfig = {config};\n'
           ) 

In [ ]:
def get_table_settings(df, title, path):
    
    data=[{'type': 'table',
           'header': {'values': [df.columns[1]], 'align': 'center', 
                      'fill': { 'color': 'paleturquoise'}},
           'cells': {'values': [df.asset_name.tolist()], 'align': 'center', 
                     'fill': { 'color': 'lavender'}}}]
    
    layout = {'title': title, 'height': 700}
    
    fig = go.Figure(data=data, layout=layout)
    fig.write_image(f'{path}/pdf/{title}.pdf')
    
    return (
            f'data = {str(data)};\n'
            f'\nlayout = {str(layout)};\n'
           ) 

In [ ]:
def html_code_pie(file_name):
    return (
f'''
    <html>
    <head>
      <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <script src="https://s3.amazonaws.com/discover.energy-XXX/{REPORT_PERIOD}/{BASIN_NAME}/js/{file_name}.js"></script>
    </head>
    <body>
      <div id='{file_name}'></div>
    </body>
        <script language="JavaScript">
            Plotly.plot('{file_name}', {{ data:data, layout:layout }});
        </script>
    </body>
    </html>
''')

In [ ]:
def html_code(file_name):
    return (
f'''
    <html>
    <head>
    <script src="https://s3.amazonaws.com/discover.energy-XXX/{REPORT_PERIOD}/{BASIN_NAME}/js/{file_name}.js"></script>
    </head>
    <body>
      <div id='{file_name}'></div>
    </body>
        <script language="JavaScript">
            Plotly.plot('{file_name}', {{ data:data, layout:layout, config:config }});
        </script>
    </body>
    </html>
''')

In [ ]:
def html_code_company(company, file_name):
    return (
f'''
    <html>
    <head>
    <script src="https://s3.amazonaws.com/discover.energy-XXX/{REPORT_PERIOD}/{BASIN_NAME}/{company}/js/{file_name}.js"></script>
    </head>
    <body>
      <div id='{file_name}'></div>
    </body>
        <script language="JavaScript">
            Plotly.plot('{file_name}', {{ data:data, layout:layout, config:config }});
        </script>
    </body>
    </html>
''')

In [ ]:
def export_to_s3(type_plot, file_name, df, column, title_plot, data=''):
    fs = s3fs.S3FileSystem(profile_name='dmitry.zorikhin@XXX')
    
    local_path = f'../reports/{REPORT_PERIOD}/{BASIN_NAME}/'
    js_path = f's3://discover.energy-XXX/{REPORT_PERIOD}/{BASIN_NAME}/js/{file_name}.js'
    with fs.open(js_path, 'wb') as f:
        if type_plot == 'pie':
            f.write(get_pie_plot_settings(
                df, column, title_plot, local_path).encode())
        elif type_plot == 'simple_bar':
            f.write(get_simple_bar_chart_settings(
                df, column, title_plot, local_path).encode())
        elif type_plot == 'bubble':
            f.write(get_bubble_chart_settings(df, title_plot, local_path).encode())
        elif type_plot == 'horizontal_stack_bar':
            f.write(get_horizontal_stack_bar_chart_settings(df, title_plot, local_path).encode())
        elif type_plot == 'stacked':
            function_return = get_stacked_chart_settings(df, title_plot, data, 
                                                         local_path, column)
            f.write(function_return.encode())
        elif type_plot == 'stacked_bar':
            function_return = get_stacked_bar_chart_settings(df, title_plot, data, 
                                                             local_path, column)
            f.write(function_return.encode())
        elif type_plot == 'grouped_bar':
            f.write(get_grouped_bar_chart_settings(df, title_plot, local_path).encode())
        elif type_plot == 'horizontal_bar':
            f.write(get_horizontal_bar_chart_settings(df, title_plot, data, local_path).encode())
        elif type_plot == 'table':
            f.write(get_table_settings(df, title_plot, local_path).encode())
    fs.chmod(js_path, 'public-read')
    
    html_path = f's3://discover.energy-XXX/{REPORT_PERIOD}/{BASIN_NAME}/html/{file_name}.html'
    with fs.open(html_path, 'wb') as f:
        if type_plot == 'pie':
            f.write(html_code_pie(file_name).encode())
        elif (type_plot == 'stacked' and function_return == 'data : none')\
          or (type_plot == 'stacked_bar' and function_return == 'data : none'):
            pass
        else:
            f.write(html_code(file_name).encode())
    fs.chmod(html_path, 'public-read')
    
    with open(f'{local_path}HTML_links_{PERMIAN_TYPE}{TODAY}.html', 'a') as html_file:
        if type_plot == 'pie':
            html_file.write(html_code_pie(file_name))
        elif (type_plot == 'stacked' and function_return == 'data : none')\
          or (type_plot == 'stacked_bar' and function_return == 'data : none'):
            pass
        else:
            html_file.write(html_code(file_name))

In [ ]:
def export_company_to_s3(company, type_plot, file_name, df, column, title_plot, data=''):    
    
    # Creating folders block
    local_path = f'../reports/{REPORT_PERIOD}/{BASIN_NAME}/{company}'
    try:
        os.mkdir(local_path)
        os.mkdir(f'{local_path}/pdf')
    except OSError as e:
        if e.errno == errno.EEXIST:
            pass
        try:
            os.mkdir(f'{local_path}/pdf')        
        except OSError as e:
            if e.errno == errno.EEXIST:
                pass
            else:
                raise
                
    fs = s3fs.S3FileSystem(profile_name='dmitry.zorikhin@XXX')
    js_path = f's3://discover.energy-XXX{REPORT_PERIOD}/{BASIN_NAME}/{company}/js/{file_name}.js'
    with fs.open(js_path, 'wb') as f:
        if type_plot == 'line':
            function_return = get_line_chart_settings(df, title_plot, data, local_path)
            f.write(function_return.encode())
        elif type_plot == 'stacked':
            function_return = get_stacked_chart_settings(df, title_plot, data, 
                                                         local_path, column)
            f.write(function_return.encode())
        elif type_plot == 'stacked_bar':
            function_return = get_stacked_bar_chart_settings(df, title_plot, data, 
                                                             local_path, column)
            f.write(function_return.encode())
        elif type_plot == 'horizontal_grouped':
            f.write(get_horizontal_grouped_bar_chart_settings(df, title_plot, 
                                                              local_path).encode())
    fs.chmod(js_path, 'public-read')
    
    html_path = f's3://discover.energy-XXX/{REPORT_PERIOD}/{BASIN_NAME}/{company}/html/{file_name}.html'
    with fs.open(html_path, 'wb') as f:
        if (type_plot == 'stacked' and function_return == 'data : none')\
          or (type_plot == 'stacked_bar' and function_return == 'data : none'):
            pass
        else:
            f.write(html_code_company(company, file_name).encode())
    fs.chmod(html_path, 'public-read')
    
    # Saving html code to paste on discover
    with open(f'../reports/{REPORT_PERIOD}/{BASIN_NAME}/{company}/HTML_links_{TODAY}.html', 'a') as html_file:
        if (type_plot == 'stacked' and function_return == 'data : none') \
          or (type_plot == 'stacked_bar' and function_return == 'data : none') \
           or (type_plot == 'line' and function_return == 'data : none'):
            pass
        else:
            html_file.write(html_code_company(company, file_name))

In [ ]:
def xlabels_format(df):
    raw_labels_list = df['quarter'].astype(str).tolist()
    xlabels_list = (['\'' + raw_labels_list[elem][2:4] + ' Q' 
                     + raw_labels_list[elem][4] for elem in range(len(raw_labels_list))])
    return xlabels_list

In [ ]:
def merge_pdf(file_name_list, name, level, company=''):
    
    if level == 'basin':
        os.chdir(f'{path}/{BASIN_NAME}/pdf')
    elif level == 'company':
        os.chdir(f'{path}/{BASIN_NAME}/{company}/pdf')
    pdfWriter = PyPDF2.PdfFileWriter()

    for filename in file_name_list:
        try:
            pdfFileObj = open(filename,'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj, strict=False)
            
            # Opening each page of the PDF
            for pageNum in range(pdfReader.numPages):
                pageObj = pdfReader.getPage(pageNum)
                pdfWriter.addPage(pageObj)

            pdfOutput = open(f'../PLOTS - {name}.pdf', 'wb')    
            pdfWriter.write(pdfOutput)    
            pdfOutput.close()
        
        except OSError as e:
            if e.errno == errno.ENOENT:
                print(f'{filename} does not exist')
            else:
                raise
    
    if level == 'basin':
        os.chdir('../../../../_scripts')
    elif level == 'company':
        os.chdir('../../../../../_scripts')

### Capex and Net Production Section

In [ ]:
start_time_to_check = tm.time()
# This required to merge pdf in the order they appear in report
capex_net_production_file_names = [] 

#### Fiscal Year 2019 Capex and Net Production per company

In [ ]:
capex_per_company = dataset_for_analysis_basin_level(assets_basins_dataset, 'capex_value', 
                                                     name_basin=BASIN_NAME, 
                                                     aggregate_basis='company', 
                                                     start_quarter=START_QUARTER_ACTUAL, 
                                                     end_quarter=END_QUARTER_ACTUAL)

In [ ]:
capex_per_company = capex_per_company.droplevel([1, 2]).reset_index()

capex_per_company['Capex ($MM)'] = capex_per_company.sum(axis=1)

capex_per_company = capex_per_company[['ticker', 'Capex ($MM)']].copy()

capex_per_company = (capex_per_company
                     .groupby(['ticker'])
                     .sum()
                     .reset_index()
                     .rename(columns={'ticker':'Company'}))
capex_per_company = capex_per_company.sort_values(by='Capex ($MM)', ascending=False)

companies_in_basin = sorted(list(capex_per_company.Company))
companies_in_basin = ', '.join(companies_in_basin)

# Metric id for net production
net_production = asset_data_df[asset_data_df['asset_metric_id'] == 162].copy() 

company_basin = (assets_basins_dataset[assets_basins_dataset['name_basin'] == BASIN_NAME]
                 [['asset_id', 'ticker']].drop_duplicates().copy())

company_net_production = (net_production
                          .merge(company_basin, left_on=['asset_id'], 
                                 right_on=['asset_id'], how='inner')[['ticker', 'value']])

company_net_production['value'] = company_net_production['value'].astype(float)
company_net_production = (company_net_production
                          .groupby(['ticker'])
                          .sum()
                          .reset_index()
                          .rename(columns={'ticker':'Company', 
                                           'value':'Net Production (Mboe/d)'}))

capex_net_production = capex_per_company.merge(company_net_production, left_on='Company', 
                                               right_on='Company', how='inner')

In [ ]:
export_to_s3('pie', f'Actual_{BASIN_NAME}_Capex_{PERMIAN_TYPE}{TODAY}', 
             capex_net_production, 'Capex ($MM)', 
             f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex ($MM)')
(capex_net_production_file_names
 .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex ($MM).pdf'))

##### Net Production per Company

In [ ]:
export_to_s3('pie', f'Actual_{BASIN_NAME}_Net_Production_{PERMIAN_TYPE}{TODAY}', 
             capex_net_production, 'Net Production (Mboe/d)',
             f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Net Production (Mboe/d)')
(capex_net_production_file_names
 .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Net Production (Mboe per d).pdf'))

##### Check 

In [ ]:
# Metric id for capex
fy_capex_from_metrics = asset_data_df[asset_data_df['asset_metric_id'] == 165].copy()

company_fy_capex_from_metrics = (fy_capex_from_metrics
                                 .merge(company_basin, left_on=['asset_id'], 
                                        right_on=['asset_id'], how='inner')[['ticker', 'value']])

company_fy_capex_from_metrics['value'] = company_fy_capex_from_metrics['value'].astype(float)
company_fy_capex_from_metrics = (company_fy_capex_from_metrics
                                 .groupby(['ticker'])
                                 .sum()
                                 .reset_index()
                                 .rename(columns={'ticker':'Company', 
                                                  'value':'Capex_metrics ($MM)'}))

capex_check = (company_fy_capex_from_metrics
               .merge(capex_per_company, left_on=['Company'], 
                      right_on=['Company'], how='inner'))
capex_check['difference'] = (round(capex_check['Capex_metrics ($MM)'] 
                                   - capex_check['Capex ($MM)'], 1))
capex_check

#### 2020 Forecasted Capex per company

In [ ]:
capex_per_company_forecasted = dataset_for_analysis_basin_level(assets_basins_dataset, 
                                                                'capex_value', 
                                                                name_basin=BASIN_NAME, 
                                                                aggregate_basis='company', 
                                                                start_quarter=START_QUARTER_FORECAST, 
                                                                end_quarter=END_QUARTER_FORECAST)

capex_per_company_forecasted = capex_per_company_forecasted.droplevel([1, 2]).reset_index()

capex_per_company_forecasted['Capex ($MM)'] = capex_per_company_forecasted.sum(axis=1)

capex_per_company_forecasted = capex_per_company_forecasted[['ticker', 'Capex ($MM)']].copy()

capex_per_company_forecasted = (capex_per_company_forecasted
                                .groupby(['ticker'])
                                .sum()
                                .reset_index()
                                .rename(columns={'ticker':'Company'}))
capex_per_company_forecasted = (capex_per_company_forecasted
                                .sort_values(by='Capex ($MM)', ascending=False))

In [ ]:
export_to_s3('pie', f'Forecasted_{BASIN_NAME}_Capex_{PERMIAN_TYPE}{TODAY}', 
             capex_per_company_forecasted, 'Capex ($MM)',
             f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex ($MM)')
(capex_net_production_file_names
 .append(f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex ($MM).pdf'))

#### Fiscal Year 2019 Capex per asset

In [ ]:
capex_per_asset = dataset_for_analysis_basin_level(assets_basins_dataset, 'capex_value', 
                                                   name_basin=BASIN_NAME, aggregate_basis='basin', 
                                                   start_quarter=START_QUARTER_ACTUAL, 
                                                   end_quarter=END_QUARTER_ACTUAL)

capex_per_asset = capex_per_asset.droplevel([0]).reset_index()

capex_per_asset['Capex ($MM)'] = capex_per_asset.sum(axis=1)

capex_per_asset['asset_name'] = '('+capex_per_asset['ticker']+') '+capex_per_asset['name']

capex_per_asset_plot = capex_per_asset[['ticker', 'name', 'asset_name', 'Capex ($MM)']].copy()
capex_per_asset = capex_per_asset[['asset_name', 'Capex ($MM)']].copy()

capex_per_asset = capex_per_asset.sort_values(by='Capex ($MM)', ascending=False)

In [ ]:
if BASIN_NAME == 'Permian' and assets_count > 30:
    # This number of assets will be displayed on separate graphs and depends on total assets 
    # (max on single graph is 35 otherwise xlabels broken)
    n = 20 
    non_zero_capex_assets_number = capex_per_asset[capex_per_asset['Capex ($MM)'] != 0].shape[0]
    actual_capex_per_asset_first_part = (capex_per_asset[capex_per_asset['Capex ($MM)'] != 0]
                                         .iloc[:n])
    actual_capex_per_asset_second_part = (capex_per_asset[capex_per_asset['Capex ($MM)'] != 0]
                                          .iloc[n:])
    zero_capex_per_asset = capex_per_asset[capex_per_asset['Capex ($MM)'] == 0]
    
    export_to_s3('simple_bar', 
                 f'{BASIN_NAME}_Actual_Capex_Allocation_by_Asset_(1st-{n-1}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 actual_capex_per_asset_first_part, 'Capex ($MM)',
                 f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation (1st-{n-1}th assets)')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation (1st-{n-1}th assets).pdf'))
    
    export_to_s3('simple_bar', 
                 f'{BASIN_NAME}_Actual_Capex_Allocation_by_Asset_({n}th-{non_zero_capex_assets_number}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 actual_capex_per_asset_second_part, 'Capex ($MM)',
                 f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation ({n}th-{non_zero_capex_assets_number}th assets)')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation ({n}th-{non_zero_capex_assets_number}th assets).pdf'))
    
    if not zero_capex_per_asset.empty:
        export_to_s3('table', 
                     f'{BASIN_NAME}_Actual_Capex_Allocation_by_Asset_(zero_capex_table)_{PERMIAN_TYPE}{TODAY}', 
                     zero_capex_per_asset, '', 
                     f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation (no capex allocated)')
        (capex_net_production_file_names
         .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation (no capex allocated).pdf'))

else:
    export_to_s3('simple_bar', 
                 f'{BASIN_NAME}_Actual_Capex_Allocation_by_Asset_{PERMIAN_TYPE}{TODAY}', 
                 capex_per_asset, 'Capex ($MM)',
                 f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation by Asset')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Actual Capex Allocation by Asset.pdf'))

#### 2020 Forecasted Capex per asset

In [ ]:
capex_per_asset_forecasted = dataset_for_analysis_basin_level(assets_basins_dataset, 
                                                              'capex_value', 
                                                              name_basin=BASIN_NAME, 
                                                              aggregate_basis='basin', 
                                                              start_quarter=START_QUARTER_FORECAST, 
                                                              end_quarter=END_QUARTER_FORECAST)

capex_per_asset_forecasted = capex_per_asset_forecasted.droplevel([0]).reset_index()

capex_per_asset_forecasted['Capex ($MM)'] = capex_per_asset_forecasted.sum(axis=1)

capex_per_asset_forecasted['asset_name'] = ('(' + capex_per_asset_forecasted['ticker']
                                            + ') ' + capex_per_asset_forecasted['name'])

capex_per_asset_forecasted = capex_per_asset_forecasted[['asset_name', 'Capex ($MM)']].copy()
capex_per_asset_forecasted = capex_per_asset_forecasted.sort_values(by='Capex ($MM)', 
                                                                    ascending=False)

In [ ]:
if BASIN_NAME == 'Permian' and assets_count > 30:
    # This number of assets will be displayed on separate graphs and depends on total assets 
    # (max on single graph is 35 otherwise xlabels broken)
    n = 20 
    non_zero_capex_assets_number_forecasted = (capex_per_asset_forecasted[
        capex_per_asset_forecasted['Capex ($MM)'] != 0].shape[0])
    actual_capex_per_asset_first_part_forecasted = (capex_per_asset_forecasted[
        capex_per_asset_forecasted['Capex ($MM)'] != 0].iloc[:n])
    actual_capex_per_asset_second_part_forecasted = (capex_per_asset_forecasted[
        capex_per_asset_forecasted['Capex ($MM)'] != 0].iloc[n:])
    zero_capex_per_asset_forecasted = (capex_per_asset_forecasted[
        capex_per_asset_forecasted['Capex ($MM)'] == 0])
    
    export_to_s3('simple_bar', 
                 f'{BASIN_NAME}_Forecasted_Capex_Allocation_by_Asset_(1st-{n-1}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 actual_capex_per_asset_first_part_forecasted, 'Capex ($MM)',
                 f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation (1st-{n-1}th assets)')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation (1st-{n-1}th assets).pdf'))
    
    export_to_s3('simple_bar', 
                 f'{BASIN_NAME}_Forecasted_Capex_Allocation_by_Asset_({n}th-{non_zero_capex_assets_number}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 actual_capex_per_asset_second_part_forecasted, 'Capex ($MM)',
                 f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation ({n}th-{non_zero_capex_assets_number_forecasted}th assets)')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation ({n}th-{non_zero_capex_assets_number_forecasted}th assets).pdf'))
    
    if not zero_capex_per_asset_forecasted.empty:
        export_to_s3('table', 
                     f'{BASIN_NAME}_Forecasted_Capex_Allocation_by_Asset_(zero_capex_table)_{TODAY}', 
                     zero_capex_per_asset_forecasted, '', 
                     f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation (no capex allocated)')
        (capex_net_production_file_names
         .append(f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation (no capex allocated).pdf'))

else:
    export_to_s3('simple_bar', 
                 f'{BASIN_NAME}_Forecasted_Capex_Allocation_by_Asset_{PERMIAN_TYPE}{TODAY}', 
                 capex_per_asset_forecasted, 'Capex ($MM)',
                 f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation by Asset')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q {BASIN_NAME} Forecasted Capex Allocation by Asset.pdf'))

#### 2019 Capex Efficiency

In [ ]:
net_production = asset_data_df[asset_data_df['asset_metric_id'] == 162].copy()

assets_basin = (assets_basins_dataset[assets_basins_dataset['name_basin'] == BASIN_NAME]
                [['asset_id', 'ticker', 'name']].drop_duplicates().copy())

assets_basin['asset_name'] = '('+assets_basin['ticker']+') '+assets_basin['name']

assets_net_production = (net_production.merge(assets_basin, left_on=['asset_id'], 
                                              right_on=['asset_id'], how='inner')
                         [['asset_name', 'value']])
assets_net_production_plot = (net_production.merge(assets_basin, left_on=['asset_id'], 
                                                   right_on=['asset_id'], how='inner')
                              [['ticker', 'name', 'asset_name', 'value']])

assets_net_production['value'] = assets_net_production['value'].astype(float)
assets_net_production_plot['value'] = assets_net_production_plot['value'].astype(float)
assets_net_production = assets_net_production.sort_values(by='value', ascending=False)

capex_efficiency = (capex_per_asset
                    .merge(assets_net_production, left_on=['asset_name'], 
                           right_on=['asset_name'], how='inner')
                    .rename(columns={'sum':'Net Capex ($MM)', 
                                     'value':'Net Production (Mboe/d)'}))
capex_efficiency_plot = (capex_per_asset_plot
                         .merge(assets_net_production_plot, left_on=['asset_name'], 
                                right_on=['asset_name'], how='inner', suffixes=('', '_y'))
                         .rename(columns={'sum':'Net Capex ($MM)', 
                                          'value':'Net Production (Mboe/d)'})
                         [['ticker', 'name', 'asset_name', 
                           'Capex ($MM)', 'Net Production (Mboe/d)']])
capex_efficiency_plot = capex_efficiency_plot.round({'Capex ($MM)': 1, 
                                                     'Net Production (Mboe/d)': 1}) 

In [ ]:
if BASIN_NAME == 'Permian' and assets_count > 30:
    capex_efficiency_plot = (capex_efficiency_plot
                             .sort_values(by='Capex ($MM)', ascending=False)
                             .dropna())
    # This number of assets will be displayed on separate graphs and depends on total assets 
    # (max on single graph is 35 otherwise xlabels broken)
    n = 20
    non_zero_capex_assets_number_efficiency = (capex_efficiency_plot[
        capex_efficiency_plot['Capex ($MM)'] != 0].shape[0])
    actual_capex_per_asset_first_part_efficiency = (capex_efficiency_plot[
        capex_efficiency_plot['Capex ($MM)'] != 0].iloc[:n])
    actual_capex_per_asset_second_part_efficiency = (capex_efficiency_plot[
        capex_efficiency_plot['Capex ($MM)'] != 0].iloc[n:])
    zero_capex_per_asset_efficiency = (capex_efficiency_plot[
        capex_efficiency_plot['Capex ($MM)'] == 0])
    zero_capex_per_asset_efficiency = (zero_capex_per_asset_efficiency
                                       .sort_values(by='Net Production (Mboe/d)', 
                                                    ascending=False))
    
    export_to_s3('bubble', 
                 f'{BASIN_NAME}_Capex_Efficiency_(1st-{n-1}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 actual_capex_per_asset_first_part_efficiency, '',
                 f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency (1st-{n-1}th assets)')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency (1st-{n-1}th assets).pdf'))
    
    export_to_s3('bubble', 
                 f'{BASIN_NAME}_Capex_Efficiency_({n}th-{non_zero_capex_assets_number}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 actual_capex_per_asset_second_part_efficiency, '',
                 f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency ({n}th-{non_zero_capex_assets_number_efficiency}th assets)')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency ({n}th-{non_zero_capex_assets_number_efficiency}th assets).pdf'))    
    
    if not zero_capex_per_asset_efficiency.empty:
        export_to_s3('horizontal_bar', 
                     f'{BASIN_NAME}_Capex_Efficiency_(zero_capex_table)_{PERMIAN_TYPE}{TODAY}', 
                     zero_capex_per_asset_efficiency, '', 
                     f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency (no capex allocated)', 'efficiency')
        (capex_net_production_file_names
         .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency (no capex allocated).pdf'))

else:
    export_to_s3('bubble', 
                 f'{BASIN_NAME}_Capex_Efficiency_{PERMIAN_TYPE}{TODAY}', 
                 capex_efficiency_plot, '',
                 f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency')
    (capex_net_production_file_names
     .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Efficiency.pdf'))

#### 2019 Capex proportion per company

In [ ]:
total_capex_proportion_per_company = dataset_for_analysis_basin_level(assets_basins_dataset, 
                                                                      'capex_value', 
                                                                      name_basin='', 
                                                                      aggregate_basis='company', 
                                                                      start_quarter=START_QUARTER_ACTUAL, 
                                                                      end_quarter=END_QUARTER_ACTUAL)

total_capex_proportion_per_company = (total_capex_proportion_per_company
                                      .droplevel([1, 2]).reset_index())

total_capex_proportion_per_company['sum'] = total_capex_proportion_per_company.sum(axis=1)

total_capex_proportion_per_company = (total_capex_proportion_per_company[['ticker', 'sum']]
                                      .copy())

total_capex_proportion_per_company = (total_capex_proportion_per_company
                                      .groupby(['ticker']).sum().reset_index())
total_capex_proportion_per_company = (total_capex_proportion_per_company
                                      .sort_values(by='sum', ascending=False))

capex_proportion_per_company = (capex_per_company
                                .merge(total_capex_proportion_per_company, left_on=['Company'], 
                                       right_on=['ticker'], how='left')
                                .rename(columns={'Capex ($MM)': BASIN_NAME, 
                                                 'sum':'total_capex'}))

capex_proportion_per_company['Other basins'] = (capex_proportion_per_company['total_capex'] 
                                                - capex_proportion_per_company[BASIN_NAME])

capex_proportion_per_company_plot = capex_proportion_per_company.copy()
(capex_proportion_per_company_plot
 [f'{BASIN_NAME}_p']) = (round((capex_proportion_per_company_plot[BASIN_NAME] 
                                / capex_proportion_per_company_plot['total_capex']), 2))
(capex_proportion_per_company_plot
 ['Other_basins_p']) = (round((capex_proportion_per_company_plot['Other basins'] 
                               / capex_proportion_per_company_plot['total_capex']), 2))

capex_proportion_per_company = capex_proportion_per_company.drop(['total_capex', 'ticker'], 
                                                                 axis=1)
capex_proportion_per_company_plot = capex_proportion_per_company_plot.drop(['total_capex', 
                                                                            'ticker'], axis=1)

In [ ]:
export_to_s3('horizontal_stack_bar', 
             f'{BASIN_NAME}_Capex_Proportion_per_Company_{PERMIAN_TYPE}{TODAY}', 
             capex_proportion_per_company_plot, '',
             f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Proportion per Company')
(capex_net_production_file_names
 .append(f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q {BASIN_NAME} Capex Proportion per Company.pdf'))

In [ ]:
capex_net_production_filename = f'{BASIN_NAME} Capex and Net Production ({companies_in_basin}) {TODAY}'

with pd.ExcelWriter(f'{path}/{BASIN_NAME}/DATA - {capex_net_production_filename}.xlsx', 
                    engine = 'xlsxwriter') as writer:
    
    capex_net_production.to_excel(writer, index=False, 
                                  sheet_name='Actual Capex and Net Production')
    worksheet = writer.sheets['Actual Capex and Net Production']
    worksheet.set_column('A:D', 20)
    
    capex_per_company_forecasted.to_excel(writer, index=False, 
                                          sheet_name = 'Forecasted Capex')
    worksheet = writer.sheets['Forecasted Capex']
    worksheet.set_column('A:D', 20)
    
    capex_per_asset.to_excel(writer, index=False, 
                             sheet_name = 'Actual Capex per asset')
    worksheet = writer.sheets['Actual Capex per asset']
    worksheet.set_column('A:D', 25)
    
    capex_per_asset_forecasted.to_excel(writer, index=False, 
                                        sheet_name = 'Forecasted Capex per asset')
    worksheet = writer.sheets['Forecasted Capex per asset']
    worksheet.set_column('A:D', 25)
    
    capex_efficiency.to_excel(writer, index=False, 
                              sheet_name = 'Actual Capex Efficiency')
    worksheet = writer.sheets['Actual Capex Efficiency']
    worksheet.set_column('A:D', 25)
    
    capex_proportion_per_company.to_excel(writer, index=False, 
                                          sheet_name = 'Actual Capex Proportion')
    worksheet = writer.sheets['Actual Capex Proportion']
    worksheet.set_column('A:D', 20)
    
    writer.save()

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    merge_pdf(capex_net_production_file_names, capex_net_production_filename, 'basin')

### Production, rig and well count per asset section

In [ ]:
production_rig_well_count_file_names = []

In [ ]:
def production(initial_dataset, parameter, name_basin, start_quarter, end_quarter):
    
    """Initial_dataset is assets_basins_dataset - as default
    parameter - either oil_value, gas_value, rig_count_value, well_count_value
    """
    
    production = dataset_for_analysis_basin_level(initial_dataset, parameter, 
                                                  name_basin=name_basin, 
                                                  aggregate_basis='company', 
                                                  start_quarter=start_quarter, 
                                                  end_quarter=end_quarter)
    production = production.droplevel([1]).reset_index()
    production['asset_name'] = '('+production['ticker']+') '+production['name']
    production = production.drop(['ticker', 'name'], axis=1)
    production = production.set_index('asset_name')
    production_transposed = production.T.reset_index()
    
    return production_transposed

In [ ]:
oil_production_basin = production(assets_basins_dataset, 'oil_value', BASIN_NAME, 
                                  START_QUARTER_ACTUAL, END_QUARTER_FORECAST)

In [ ]:
export_to_s3('stacked', f'{BASIN_NAME}_Oil_Production_{PERMIAN_TYPE}{TODAY}', 
             oil_production_basin, 'general', f'{BASIN_NAME} Oil Production', 'oil')
production_rig_well_count_file_names.append(f'{BASIN_NAME} Oil Production.pdf')

In [ ]:
gas_production_basin = production(assets_basins_dataset, 'gas_value', BASIN_NAME, 
                                  START_QUARTER_ACTUAL, END_QUARTER_FORECAST)

In [ ]:
export_to_s3('stacked', f'{BASIN_NAME}_Gas_Production_{PERMIAN_TYPE}{TODAY}', 
             gas_production_basin, 'general', f'{BASIN_NAME} Gas Production', 'gas')
production_rig_well_count_file_names.append(f'{BASIN_NAME} Gas Production.pdf')

In [ ]:
rig_count_value_basin = production(assets_basins_dataset, 'rig_count_value', BASIN_NAME, 
                                   START_QUARTER_ACTUAL, END_QUARTER_FORECAST)

In [ ]:
export_to_s3('stacked_bar', f'{BASIN_NAME}_Active_Rigs_{PERMIAN_TYPE}{TODAY}', 
             rig_count_value_basin, 'general', f'{BASIN_NAME} Active Rigs', 'rig')
production_rig_well_count_file_names.append(f'{BASIN_NAME} Active Rigs.pdf')

In [ ]:
well_count_value_per_quarter = production(assets_basins_dataset, 'well_count_value', 
                                          BASIN_NAME, START_QUARTER_ACTUAL, 
                                          END_QUARTER_FORECAST)

In [ ]:
export_to_s3('stacked_bar', 
             f'{BASIN_NAME}_New_Wells_Brought_Online_Quarterly_{PERMIAN_TYPE}{TODAY}', 
             well_count_value_per_quarter, 'general', 
             f'{BASIN_NAME} New Wells Brought Online Quarterly', 'well')
(production_rig_well_count_file_names
 .append(f'{BASIN_NAME} New Wells Brought Online Quarterly.pdf'))

##### Fact / Forecast Basin New Wells Brought Online

In [ ]:
new_wells_fact = dataset_for_analysis_basin_level(assets_basins_dataset, 'well_count_value', 
                                                  name_basin=BASIN_NAME, aggregate_basis='basin', 
                                                  start_quarter=START_QUARTER_ACTUAL, 
                                                  end_quarter=END_QUARTER_ACTUAL)

new_wells_fact = new_wells_fact.droplevel([0, 2]).reset_index()
new_wells_fact = new_wells_fact.groupby('ticker').sum().reset_index()
new_wells_fact_period_name = f'{START_QUARTER_ACTUAL}Q-{END_QUARTER_ACTUAL}Q fact'
new_wells_fact[new_wells_fact_period_name] = new_wells_fact.sum(axis=1)
new_wells_fact = new_wells_fact[['ticker', new_wells_fact_period_name]].copy()
new_wells_fact = new_wells_fact.sort_values(by=new_wells_fact_period_name, ascending=False)

In [ ]:
new_wells_forecast = dataset_for_analysis_basin_level(assets_basins_dataset, 'well_count_value', 
                                                      name_basin=BASIN_NAME, 
                                                      aggregate_basis='basin', 
                                                      start_quarter=START_QUARTER_FORECAST, 
                                                      end_quarter=END_QUARTER_FORECAST)

new_wells_forecast = new_wells_forecast.droplevel([0, 2]).reset_index()
new_wells_forecast = new_wells_forecast.groupby('ticker').sum().reset_index()
new_wells_forecast_period_name = f'{START_QUARTER_FORECAST}Q-{END_QUARTER_FORECAST}Q forecast'
new_wells_forecast[new_wells_forecast_period_name] = new_wells_forecast.sum(axis=1)
new_wells_forecast = new_wells_forecast[['ticker', new_wells_forecast_period_name]].copy()
new_wells_forecast = (new_wells_forecast
                      .sort_values(by=new_wells_forecast_period_name, ascending=False))

In [ ]:
new_wells_plan_forecast = new_wells_fact.merge(new_wells_forecast, left_on='ticker', 
                                               right_on='ticker', how='left')

In [ ]:
export_to_s3('grouped_bar', 
             f'Fact_Forecast_{BASIN_NAME}_New_Wells_Brought_Online_{PERMIAN_TYPE}{TODAY}', 
             new_wells_plan_forecast, '', f'Fact / Forecast {BASIN_NAME} New Wells Brought Online')
(production_rig_well_count_file_names
 .append(f'Fact and Forecast {BASIN_NAME} New Wells Brought Online.pdf'))

##### Lateral length / feet between wells

In [ ]:
def lateral_length_feet_between_wells(asset_data_dataset, initial_dataset, 
                                      asset_metric_id, name_basin, parameter):
    
    """
    asset_data_dataset should be asset_data_df
    initial_dataset is assets_basins_dataset - as default
    integer value asset_metric_id either 11 for lateral length or 12 for feet between wells
    parameter - 'assets_level' or 'company_level'
    """
    
    metric_dataset = asset_data_dataset[asset_data_dataset['asset_metric_id'] == asset_metric_id].copy()
    assets_basin = (initial_dataset[initial_dataset['name_basin'] == name_basin]
                    [['asset_id', 'ticker', 'name']].drop_duplicates().copy())
    assets_basin['asset_name'] = '('+assets_basin['ticker']+') '+assets_basin['name']
    
    if parameter == 'assets_level':
        assets_metric = (metric_dataset.merge(assets_basin, left_on=['asset_id'], 
                                              right_on=['asset_id'], how='inner')
                         [['asset_name', 'value']])
    elif parameter == 'company_level':
        assets_metric = (metric_dataset
                         .merge(assets_basin, left_on=['asset_id'], 
                                right_on=['asset_id'], how='inner'))
    assets_metric['value'] = assets_metric['value'].astype(int)
    assets_metric = assets_metric.sort_values(by='value', ascending=False)
    
    return assets_metric

In [ ]:
assets_lateral_length = lateral_length_feet_between_wells(asset_data_df, assets_basins_dataset, 
                                                          11, BASIN_NAME, 'assets_level')

In [ ]:
if BASIN_NAME == 'Permian' and assets_count > 70:
    n = 45 # first n assets to display
    assets_number = assets_lateral_length.shape[0]
    assets_lateral_length_first_part = assets_lateral_length.iloc[:n]
    assets_lateral_length_second_part = assets_lateral_length.iloc[n:]
    
    export_to_s3('horizontal_bar', 
                 f'{BASIN_NAME}_Asset_Wells_Lateral_Length_(first_{n-1}_assets)_{PERMIAN_TYPE}{TODAY}', 
                 assets_lateral_length_first_part, '',
                 f'{BASIN_NAME} Asset Wells Lateral Length (first {n-1} assets)', 'lateral')
    (production_rig_well_count_file_names
     .append(f'{BASIN_NAME} Asset Wells Lateral Length (first {n-1} assets).pdf'))
    
    export_to_s3('horizontal_bar', 
                 f'{BASIN_NAME}_Asset_Wells_Lateral_Length_({n}th-{assets_number}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 assets_lateral_length_second_part, '',
                 f'{BASIN_NAME} Asset Wells Lateral Length ({n}th-{assets_number}th assets)', 
                 'lateral')
    (production_rig_well_count_file_names
     .append(f'{BASIN_NAME} Asset Wells Lateral Length ({n}th-{assets_number}th assets).pdf'))
else:
    export_to_s3('horizontal_bar', 
                 f'{BASIN_NAME}_Asset_Wells_Lateral_Length_{PERMIAN_TYPE}{TODAY}', 
                 assets_lateral_length, '',
                 f'{BASIN_NAME} Asset Wells Lateral Length', 'lateral')
    production_rig_well_count_file_names.append(f'{BASIN_NAME} Asset Wells Lateral Length.pdf')

In [ ]:
assets_feet_between_wells = lateral_length_feet_between_wells(asset_data_df, 
                                                              assets_basins_dataset, 
                                                              12, BASIN_NAME, 'assets_level')

In [ ]:
if BASIN_NAME == 'Permian' and assets_count > 70:
    n = 45 # first n assets to display
    assets_number = assets_feet_between_wells.shape[0]
    assets_feet_between_wells_first_part = assets_feet_between_wells.iloc[:n]
    assets_feet_between_wells_second_part = assets_feet_between_wells.iloc[n:]
    
    export_to_s3('horizontal_bar', 
                 f'{BASIN_NAME}_Feet_Between_Wells_(first_{n-1}_assets)_{PERMIAN_TYPE}{TODAY}', 
                 assets_feet_between_wells_first_part, '',
                 f'{BASIN_NAME} Feet Between Wells (first {n-1} assets)', 'distance')
    production_rig_well_count_file_names.append(f'{BASIN_NAME} Feet Between Wells (first {n-1} assets).pdf')
    
    export_to_s3('horizontal_bar', 
                 f'{BASIN_NAME}_Feet_Between_Wells_({n}th-{assets_number}th_assets)_{PERMIAN_TYPE}{TODAY}', 
                 assets_feet_between_wells_second_part, '',
                 f'{BASIN_NAME} Feet Between Wells ({n}th-{assets_number}th assets)', 'distance')
    (production_rig_well_count_file_names
     .append(f'{BASIN_NAME} Feet Between Wells ({n}th-{assets_number}th assets).pdf'))
else:    
    export_to_s3('horizontal_bar', 
                 f'{BASIN_NAME}_Feet_Between_Wells_{PERMIAN_TYPE}{TODAY}', 
                 assets_feet_between_wells, '',
                 f'{BASIN_NAME} Feet Between Wells', 'distance')
    production_rig_well_count_file_names.append(f'{BASIN_NAME} Feet Between Wells.pdf')

In [ ]:
assets_lateral_length_feet_between_wells = (assets_lateral_length
                                            .merge(assets_feet_between_wells, 
                                                   left_on=['asset_name'], 
                                                   right_on=['asset_name'], 
                                                   how='inner')
                                            .rename(columns={'value_x':'Lateral length, ft', 
                                                             'value_y':'Feet between wells, ft'})
                                            [['asset_name', 'Lateral length, ft', 
                                              'Feet between wells, ft']])

In [ ]:
production_rigs_wells_filename = f'{BASIN_NAME} Production, New Wells and Rigs ({companies_in_basin}) {TODAY}'

with pd.ExcelWriter(f'{path}/{BASIN_NAME}/DATA - {production_rigs_wells_filename}.xlsx', engine = 'xlsxwriter') as writer:
    
    oil_production_basin.to_excel(writer, index=False, sheet_name='Oil Production')
    worksheet = writer.sheets['Oil Production']
    worksheet.set_column('A:BA', 25)
    
    gas_production_basin.to_excel(writer, index=False, sheet_name = 'Gas Production')
    worksheet = writer.sheets['Gas Production']
    worksheet.set_column('A:BA', 25)
    
    rig_count_value_basin.to_excel(writer, index=False, sheet_name = 'Active Rigs')
    worksheet = writer.sheets['Active Rigs']
    worksheet.set_column('A:BA', 25)
    
    well_count_value_per_quarter.to_excel(writer, index=False, 
                                          sheet_name = 'Wells Brought Online Quarter')
    worksheet = writer.sheets['Wells Brought Online Quarter']
    worksheet.set_column('A:BA', 30)
    
    new_wells_plan_forecast.to_excel(writer, index=False, 
                                     sheet_name = 'Fact and Forecast Wells')
    worksheet = writer.sheets['Fact and Forecast Wells']
    worksheet.set_column('A:D', 20)
    
    assets_lateral_length.to_excel(writer, index=False, 
                                   sheet_name = 'Asset Wells Lateral Length')
    worksheet = writer.sheets['Asset Wells Lateral Length']
    worksheet.set_column('A:D', 30)
    
    assets_feet_between_wells.to_excel(writer, index=False, 
                                       sheet_name = 'Distance Between Asset Wells')
    worksheet = writer.sheets['Distance Between Asset Wells']
    worksheet.set_column('A:D', 30)
    
    (assets_lateral_length_feet_between_wells
     .to_excel(writer, index=False, sheet_name = 'Lat Length Dist Btw Asset Wells'))
    worksheet = writer.sheets['Lat Length Dist Btw Asset Wells']
    worksheet.set_column('A:D', 30)
    
    writer.save()

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    merge_pdf(production_rig_well_count_file_names, production_rigs_wells_filename, 'basin')

### Company level analysis

In [ ]:
basin_level_analytics_time = round((tm.time() - start_time_to_check), 1)
print(f'--- High level basin analysis took {basin_level_analytics_time} seconds ---')

In [ ]:
def company_level_analysis(initial_dataset, company, basin, 
                           parameter, start_quarter, end_quarter):
    
    """
    initial_dataset is assets_basins_dataset - as default
    company - company ticker
    basin - BASIN_NAME
    parameter - 'type_curve_oil_value' ; 'oil_value' ; 'type_curve_gas_value' ; 'gas_value' ; 
    'rig_count_value' ; 'well_count_value' ; 'capex_value'
    """
    
    df = (initial_dataset[(initial_dataset['ticker'] == company) 
                          & (initial_dataset['name_basin'] == basin) 
                          & (initial_dataset['quarter'] >= start_quarter) 
                          & (initial_dataset['quarter'] <= end_quarter)])
    assets = list(df.name.unique())

    if parameter == 'type_curve_oil_value' or parameter == 'type_curve_gas_value':
        dataset_parameter = df[(df[parameter] > 0) & (df['name'] == assets[0])][['quarter']]
        
        for asset in assets:
            if df[df['name'] == asset][parameter].sum() == 0:
                dataset_parameter[asset] = np.nan
            else:
                dataset_parameter[asset] = (df[(df[parameter] > 0) 
                                               & (df['name'] == asset)][parameter].values)
                dataset_parameter['quarter'] = (df[(df[parameter] > 0) 
                                                   & (df['name'] == asset)]['quarter'].values)
            
    else:
        dataset_parameter = df[(df['name'] == assets[0])][['quarter']]
        
        for asset in assets:
            dataset_parameter[asset] = df[(df['name'] == asset)][parameter].values
        
    return dataset_parameter

In [ ]:
start_time_to_check_company = tm.time()

company_lateral_length = lateral_length_feet_between_wells(asset_data_df, 
                                                           assets_basins_dataset, 
                                                           11, BASIN_NAME, 'company_level')
company_feet_between_wells = lateral_length_feet_between_wells(asset_data_df, 
                                                               assets_basins_dataset, 
                                                               12, BASIN_NAME, 'company_level')
company_assets_lateral_length_feet_between_wells = (company_lateral_length
                                                    .merge(company_feet_between_wells, 
                                                           left_on=['asset_name'], 
                                                           right_on=['asset_name'], 
                                                           how='inner')
                                                    .rename(columns={
                                                        'value_x':'Lateral length, ft', 
                                                        'value_y':'Feet between wells, ft', 
                                                        'ticker_x':'Company', 
                                                        'name_x':'asset'})
                                                    [['Company', 'asset', 
                                                      'Lateral length, ft', 
                                                      'Feet between wells, ft']])

for company in list(capex_per_company.Company):
    
    company_file_names = []
    
    type_curve_oil = company_level_analysis(assets_basins_dataset, company, BASIN_NAME, 
                                            'type_curve_oil_value', START_QUARTER_ACTUAL, 20234)

    export_company_to_s3(company, 'line', f'{company}_{BASIN_NAME}_Oil_Type_Curve_{TODAY}', 
                 type_curve_oil, '', f'{company} - {BASIN_NAME} Oil Type Curve', 'oil')
    company_file_names.append(f'{company} - {BASIN_NAME} Oil Type Curve.pdf')
    
    oil_production = company_level_analysis(assets_basins_dataset, company, 
                                            BASIN_NAME, 'oil_value', 
                                            START_QUARTER_ACTUAL, END_QUARTER_FORECAST)
    
    export_company_to_s3(company, 'stacked', f'{company}_{BASIN_NAME}_Oil_Production_{TODAY}', 
                     oil_production, '', f'{company} - {BASIN_NAME} Oil Production', 'oil')
    company_file_names.append(f'{company} - {BASIN_NAME} Oil Production.pdf')

    type_curve_gas = company_level_analysis(assets_basins_dataset, company, BASIN_NAME, 
                                            'type_curve_gas_value', START_QUARTER_ACTUAL, 20234)
    
    export_company_to_s3(company, 'line', f'{company}_{BASIN_NAME}_Gas_Type_Curve_{TODAY}', 
                 type_curve_gas, '', f'{company} - {BASIN_NAME} Gas Type Curve', 'gas')
    company_file_names.append(f'{company} - {BASIN_NAME} Gas Type Curve.pdf')

    gas_production = company_level_analysis(assets_basins_dataset, company, 
                                            BASIN_NAME, 'gas_value', 
                                            START_QUARTER_ACTUAL, END_QUARTER_FORECAST)
    
    export_company_to_s3(company, 'stacked', f'{company}_{BASIN_NAME}_Gas_Production_{TODAY}', 
                     gas_production, '', f'{company} - {BASIN_NAME} Gas Production', 'gas')
    company_file_names.append(f'{company} - {BASIN_NAME} Gas Production.pdf')

    rig_count = (company_level_analysis(assets_basins_dataset, company, 
                                        BASIN_NAME, 'rig_count_value', 
                                        START_QUARTER_ACTUAL, END_QUARTER_FORECAST))
    
    export_company_to_s3(company, 'stacked_bar', f'{company}_{BASIN_NAME}_Active_Rigs_{TODAY}', 
                     rig_count, '', f'{company} - {BASIN_NAME} Active Rigs', 'rig')
    company_file_names.append(f'{company} - {BASIN_NAME} Active Rigs.pdf')

    well_count_value = (company_level_analysis(assets_basins_dataset, company, 
                                               BASIN_NAME, 'well_count_value', 
                                               START_QUARTER_ACTUAL, END_QUARTER_FORECAST))
    
    export_company_to_s3(company, 'stacked_bar', 
                         f'{company}_{BASIN_NAME}_New_Wells_Brought_Online_{TODAY}', 
                         well_count_value, '', 
                         f'{company} - {BASIN_NAME} New Wells Brought Online', 'well')
    company_file_names.append(f'{company} - {BASIN_NAME} New Wells Brought Online.pdf')

    capex_value = (company_level_analysis(assets_basins_dataset, company, 
                                          BASIN_NAME, 'capex_value', 
                                          START_QUARTER_ACTUAL, END_QUARTER_FORECAST))
    
    export_company_to_s3(company, 'stacked_bar', f'{company}_{BASIN_NAME}_Capex_{TODAY}', 
                     capex_value, '', f'{company} - {BASIN_NAME} Capex', 'capex')
    company_file_names.append(f'{company} - {BASIN_NAME} Capex.pdf')

    company_level_filename = f'{company} - {BASIN_NAME} Analysis {TODAY}'
    
    lat_length_feet_distance = (company_assets_lateral_length_feet_between_wells[
        company_assets_lateral_length_feet_between_wells['Company'] == company]
                                [['asset', 'Lateral length, ft', 'Feet between wells, ft']])
    
    export_company_to_s3(company, 
                         'horizontal_grouped', 
                         f'{company}_{BASIN_NAME}_Lateral_Length_and_Feet_Between_Wells_{TODAY}', 
                         lat_length_feet_distance, '', 
                         f'{company} - {BASIN_NAME} Lateral Length and Feet Between Wells')
    (company_file_names
     .append(f'{company} - {BASIN_NAME} Lateral Length and Feet Between Wells.pdf'))
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        merge_pdf(company_file_names, company_level_filename, 'company', company)

    with pd.ExcelWriter(f'{path}/{BASIN_NAME}/{company}/DATA - {company} - {BASIN_NAME} Analysis.xlsx', 
                        engine = 'xlsxwriter') as writer:

        type_curve_oil.to_excel(writer, index=False, sheet_name='Oil Type Curve')
        worksheet = writer.sheets['Oil Type Curve']
        worksheet.set_column('A:BA', 25)

        oil_production.to_excel(writer, index=False, sheet_name = 'Oil Production')
        worksheet = writer.sheets['Oil Production']
        worksheet.set_column('A:BA', 25)

        type_curve_gas.to_excel(writer, index=False, sheet_name = 'Gas Type Curve')
        worksheet = writer.sheets['Gas Type Curve']
        worksheet.set_column('A:BA', 25)

        gas_production.to_excel(writer, index=False, sheet_name = 'Gas Production')
        worksheet = writer.sheets['Gas Production']
        worksheet.set_column('A:BA', 25)

        rig_count.to_excel(writer, index=False, sheet_name = 'Active Rigs')
        worksheet = writer.sheets['Active Rigs']
        worksheet.set_column('A:BA', 25)

        well_count_value.to_excel(writer, index=False, sheet_name = 'New Wells Brought Online')
        worksheet = writer.sheets['New Wells Brought Online']
        worksheet.set_column('A:BA', 25)

        capex_value.to_excel(writer, index=False, sheet_name = 'Capex ($MM)')
        worksheet = writer.sheets['Capex ($MM)']
        worksheet.set_column('A:BA', 25)
        
        lat_length_feet_distance.to_excel(writer, index=False, sheet_name = 'Lat Length Dist Btw Asset Wells')
        worksheet = writer.sheets['Lat Length Dist Btw Asset Wells']
        worksheet.set_column('A:BA', 25)

        writer.save()

company_level_analytics_time = round((tm.time() - start_time_to_check_company), 1)

In [ ]:
print(f'--- High level basin analysis took {basin_level_analytics_time} seconds ---')
print(f'--- Company level analysis took {company_level_analytics_time} seconds ---')
print(f'--- Total time for analysis {basin_level_analytics_time + company_level_analytics_time} seconds ---')